# PNOF7s

In [19]:
import yaml
import numpy as np

set_name = "050"
nof = "PNOF7s"
ncwo = "ncwomax"
file = open('{}-{}-{}.yaml'.format(set_name,nof,ncwo), 'r')
data = yaml.safe_load(file)

In [20]:
def get_MADs(data):
    MADs = []
    for set in data:
        ADs = []
        for system in data[set]:
            ADs.append(data[set][system]["AD"])
        MADs.append(np.mean(ADs))
    return MADs

def get_Sets(data):
    Sets = []
    for set in data:
        Sets.append(set)
    return Sets

def get_Descriptions(subsets):
    f_des = open('Sets_description.yaml', 'r')
    d_des = yaml.safe_load(f_des)
    descriptions = []
    for set in subsets:
        descriptions.append(d_des[set])
    return descriptions

In [21]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [22]:
import math
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, TapTool
from bokeh.palettes import RdYlGn
from bokeh.transform import linear_cmap

subsets = get_Sets(data)
counts = get_MADs(data)
description = get_Descriptions(subsets)

source = ColumnDataSource(data=dict(subsets=subsets, counts=counts, description=description))

p = figure(x_range=subsets, height=400, title="DietGMTKN55 {} {} Mean Absolute Deviation".format(set_name, nof))

p.xaxis.major_label_orientation = math.pi/2

p.vbar(x='subsets', top='counts', width=0.9, source=source,
       fill_color=linear_cmap('counts', palette=RdYlGn[11], low=min(counts), high=max(counts)), 
        line_color="white")

#p.add_tools(TapTool())
hover = HoverTool(tooltips=[
    ("Description", "@description"),
    ("MAD", "@counts"),
])
p.add_tools(hover)

show(p)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import py3Dmol

master_tabs = widgets.Tab()

tabs = []
for h,set in enumerate(data):
    set_tab = widgets.Tab()
    set_tab.children = [widgets.VBox() for system in data[set]]
    for i,system in enumerate(data[set]):
        set_tab.set_title(i, str(system))

        mols_tab = widgets.Tab()
        mols_tab.children = [widgets.VBox() for specie in data[set][system]["Species"]]
        childrens = []
        for j,specie in enumerate(data[set][system]["Species"]):
            url = "url:https://raw.githubusercontent.com/felipelewyee/DietGMTKN55-NOF/refs/heads/main/fig/{}/{}-{}.xyz".format(set_name, set, specie) 
            viewer = py3Dmol.view(url, width=400, height=300)
            viewer.setStyle({'stick': {'radius': .1}, 'sphere': {'scale': 0.25}})
            viewer.zoomTo()
            childrens.append(widgets.Output()) 
            with childrens[-1]:
                viewer.show()
        mols_tab.children = childrens
        for j,specie in enumerate(data[set][system]["Species"]):
            mols_tab.set_title(j, str(specie))

        text_specie = "| Specie | E_NOF | Count |\n"
        for specie in data[set][system]["Species"]:
            text_specie += "| {} | {:.3f} | {} |\n".format(specie, data[set][system]["Species"][specie]["E_NOF"], 
                                                data[set][system]["Species"][specie]["Count"])
                                                #data[set][system]["Species"][specie]["Charge"], 
                                                #data[set][system]["Species"][specie]["Multiplicity"])
 
        text_specie_w = widgets.Textarea(
            value=text_specie,
            placeholder='Type something',
            description='Molecules',
            disabled=True   
        )

        text_E_NOF_w = widgets.Textarea(
            value="{:.3f}".format(data[set][system]["Energy"]*627.15),
            placeholder='Type something',
            description='E_NOF',
            disabled=True   
        )

        text_E_ref_w = widgets.Textarea(
            value="{:.3f}".format(data[set][system]["E_Ref"]),
            placeholder='Type something',
            description='E_Ref',
            disabled=True   
        )

        text_AD_w = widgets.Textarea(
            value="{:.3f}".format(data[set][system]["AD"]),
            placeholder='Type something',
            description='AD',
            disabled=True   
        )

        text_weight_w = widgets.Textarea(
            value="{:.3f}".format(data[set][system]["Weight"]),
            placeholder='Type something',
            description='Weight',
            disabled=True   
        )

        text_WTMAD2_w = widgets.Textarea(
            value="{:.3f}".format(data[set][system]["WTMAD2"]),
            placeholder='Type something',
            description='WTMAD2',
            disabled=True   
        )

        set_tab.children[i].children = [mols_tab, text_specie_w,
                                         text_E_NOF_w, text_E_ref_w, 
                                         text_AD_w, text_weight_w,
                                         text_WTMAD2_w]
    tabs.append(set_tab)

master_tabs.children = tabs
for i,set in enumerate(data):
    master_tabs.set_title(i, str(set))

display(master_tabs)
